## Group Project Exploratory Notebook 2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
import os
import plotly.offline as po
import plotly.graph_objs as pg
from functools import reduce
from scipy.stats import zscore

Load Food Environment Atlas from usda. The Excel fiels comes with multiple spreadsheets with categories health, insecurity, socioeconomic, stores, local, and access. 

In [3]:
health = pd.read_excel('usda_health.xlsx')
insecurity = pd.read_excel('usda_insecurity.xlsx')
socioeconomic = pd.read_excel('usda_socioeconomic.xlsx')
stores = pd.read_excel('usda_stores.xlsx')
access = pd.read_excel('usda_access.xlsx')

Retain only the variables that we want for merging, building the index, and outcomes.

In [4]:
health_df = health[["FIPS", "State", "County", 
                 "PCT_DIABETES_ADULTS19", 
                "PCT_OBESE_ADULTS22"]]

In [6]:
insecurity_df = insecurity[["FIPS", "State", "County", 
                "FOODINSEC_18_20", "VLFOODSEC_18_20"]]

In [23]:
socio_df = socioeconomic[[
    "FIPS",
    "State",
    "County",
    "MEDHHINC21",
    "POVRATE21",
    "DEEPPOVRATE21",
    "CHILDPOVRATE21",
    "DEEPCHILDPOVRATE21",
    "PCT_65OLDER20"
]]

In [10]:
stores_df = stores[[
    "FIPS",
    "State",
    "County",
    "GROCPTH20",
    "SUPERCPTH20",
    "CONVSPTH20",
    "SPECSPTH20",
    "SNAPSPTH23",
    "WICSPTH22"
]]

In [25]:
access_df = access[['FIPS', 'State', 'County', 'PCT_LACCESS_POP19', 'PCT_LACCESS_LOWI19', 'PCT_LACCESS_HHNV19', 'PCT_LACCESS_SNAP19', 'PCT_LACCESS_CHILD19', 'PCT_LACCESS_SENIORS19']]


Combine the data frames

In [1]:
dfs = [health_df, insecurity_df, socio_df, stores_df, access_df]

NameError: name 'health_df' is not defined

In [ ]:
df = reduce(lambda left, right: pd.merge(left, right, on=['FIPS', 'State', 'County'], how='left'), dfs)

In [ ]:
df.info()

Now rename the variables into something easier to understand.

In [ ]:
rename = {
    "PCT_DIABETES_ADULTS19": "diabetes",
    "PCT_OBESE_ADULTS22": "obesity",
    "FOODINSEC_18_20": "food_insecurity",
    "VLFOODSEC_18_20": "very_low_food_security",
    "MEDHHINC21": "median_income",
    "POVRATE21": "poverty_rate",
    "DEEPPOVRATE21": "deep_poverty_rate",
    "CHILDPOVRATE21": "child_poverty_rate",
    "DEEPCHILDPOVRATE21": "deep_child_poverty_rate",
    "PCT_65OLDER20": "senior_population_pct",
    "GROCPTH20": "grocery_stores",
    "SUPERCPTH20": "supercenters",
    "CONVSPTH20": "convenience_stores",
    "SPECSPTH20": "specialty_stores",
    "SNAPSPTH23": "snap_stores",
    "WICSPTH22": "wic_stores",
    "PCT_LACCESS_POP19": "low_access_population_pct",
    "PCT_LACCESS_LOWI19": "low_income_low_access_pct",
    "PCT_LACCESS_HHNV19": "no_vehicle_low_access_pct",
    "PCT_LACCESS_SNAP19": "snap_low_access_pct",
    "PCT_LACCESS_CHILD19": "child_low_access_pct",
    "PCT_LACCESS_SENIORS19": "senior_low_access_pct"
}

df = df.rename(columns=rename)


In [ ]:
df.describe()

There are -9999, -8888 values in places that don't make sense. When I looked around online it was said this can be used as a placeholder for missing data. I replaced them with NaN.

In [ ]:
df.replace([-9999, -8888], np.nan, inplace=True)

In [ ]:
df.describe()

In [ ]:
df.info()

I chose to do mean imputation at the state level

In [ ]:
impute = [c for c in df.columns if c not in ['FIPS', 'State', 'County'] and pd.api.types.is_numeric_dtype(df[c])]

In [ ]:
df[impute] = df.groupby('State')[impute].transform(lambda x: x.fillna(x.mean()))

In [ ]:
df.info()

Sorry, I know this is redundant, but I decided to drop columns after deciding what to keep for the index. df_index becomes the new data frame

In [ ]:
df_index = df[['FIPS', 'State', 'County',
               'diabetes', 'obesity',
               'poverty_rate', 'deep_poverty_rate', 'child_poverty_rate', 'median_income',
               'low_access_population_pct', 'low_income_low_access_pct', 'no_vehicle_low_access_pct',
               'child_low_access_pct', 'senior_low_access_pct',
                'supercenters', 'grocery_stores']].copy()

## Create an index
Create the index with the following variables:

Variable,	Description
- FIPS,	County FIPS code identifying each county.
- State,	State name.
- County,	County name.
- diabetes,	Percentage of adults diagnosed with diabetes (2019).
- obesity,	Percentage of adults classified as obese (2022).
- poverty_rate,	Percentage of individuals living below the poverty line (2021).
- deep_poverty_rate,	Percentage of individuals living in deep poverty (2021).
- child_poverty_rate,	Percentage of children living below the poverty line (2021).
- median_income,	Median household income in dollars (2021).
- low_access_population_pct,	Percentage of total population with low access to grocery stores (2019).
- low_income_low_access_pct,	Percentage of low-income population with low access to grocery stores (2019).
- no_vehicle_low_access_pct,	Percentage of households without a vehicle and with low food access (2019).
- child_low_access_pct,	Percentage of children with low access to grocery stores (2019).
- senior_low_access_pct,	Percentage of seniors (65+) with low access to grocery stores (2019).
- supercenters,	Number of supercenters and warehouse clubs per 1,000 residents (2020).
- grocery_stores,	Number of grocery stores per 1,000 residents (2020).

Create the index. Start by inverting the variables where higher values would create better conditions for food access. Then standardize the selected variables by using z-score. Then compute the mean z-score across all variables to get the index.

In [ ]:
invert_vars = ['median_income', 'low_access_population_pct', 'low_income_low_access_pct',
               'no_vehicle_low_access_pct', 'child_low_access_pct', 'senior_low_access_pct',
                'supercenters', 'grocery_stores']

for col in invert_vars:
    df_index[col + '_inv'] = -df_index[col]

index_vars = ['poverty_rate', 'deep_poverty_rate', 'child_poverty_rate'] + [col + '_inv' for col in invert_vars]

df_index['Food_Desert_Index'] = df_index[index_vars].apply(zscore).mean(axis=1)


In [ ]:
df_index

Now create a scale 1-5 for the food desert index

In [ ]:
df_index['FDI_Category'] = pd.qcut(df_index['Food_Desert_Index'], 5, labels=[1, 2, 3, 4, 5])

In [ ]:
df_index.info()

In [ ]:
df_index.describe()